In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 40*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240401150026
0	Validation losses: 0.4432, 25.6610, 26.1042	Best Loss: 26.1042 (0)	Accuracy: 83.18%
1	Validation losses: 0.3969, 11.5795, 11.9764	Best Loss: 11.9764 (0)	Accuracy: 86.26%
2	Validation losses: 0.4212, 3.1580, 3.5792	Best Loss: 3.5792 (0)	Accuracy: 84.80%
3	Validation losses: 0.4739, 0.7114, 1.1854	Best Loss: 1.1854 (0)	Accuracy: 82.82%
4	Validation losses: 0.4402, 0.3272, 0.7674	Best Loss: 0.7674 (0)	Accuracy: 83.86%
5	Validation losses: 0.5219, 0.2753, 0.7972	Best Loss: 0.7674 (1)	Accuracy: 80.44%
6	Validation losses: 0.4089, 0.2603, 0.6692	Best Loss: 0.6692 (0)	Accuracy: 85.98%
7	Validation losses: 0.5727, 0.2513, 0.8240	Best Loss: 0.6692 (1)	Accuracy: 78.40%
8	Validation losses: 0.4598, 0.2498, 0.7095	Best Loss: 0.6692 (2)	Accuracy: 84.18%
9	Validation losses: 0.5670, 0.2301, 0.7972	Best Loss: 0.6692 (3)	Accuracy: 77.20%
10	Validation losses: 0.5288, 0.2116, 0.7404	Best Loss: 0.6692 (4)	Accuracy: 82.20%
11	Validation losses: 0

28	Validation losses: 2.3044, 0.1144, 2.4188	Best Loss: 0.6290 (14)	Accuracy: 9.80%
29	Validation losses: 2.3041, 0.1139, 2.4181	Best Loss: 0.6290 (15)	Accuracy: 10.08%
30	Validation losses: 2.3037, 0.1135, 2.4172	Best Loss: 0.6290 (16)	Accuracy: 10.08%
31	Validation losses: 2.3028, 0.1131, 2.4158	Best Loss: 0.6290 (17)	Accuracy: 11.12%
32	Validation losses: 2.3039, 0.1127, 2.4167	Best Loss: 0.6290 (18)	Accuracy: 9.14%
33	Validation losses: 2.3034, 0.1124, 2.4158	Best Loss: 0.6290 (19)	Accuracy: 10.02%
34	Validation losses: 2.3039, 0.1121, 2.4160	Best Loss: 0.6290 (20)	Accuracy: 9.80%
Early stopping!
Total running time:	 2803
INFO:tensorflow:Restoring parameters from ./models/model_20240401162503.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    18.0599999428%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.2
Now: 20240401171147
0	Validation losses: 0.4196, 18.3660, 18.7856	Best Loss: 18.7856 (0)	Acc

56	Validation losses: 0.4180, 0.0898, 0.5079	Best Loss: 0.4663 (16)	Accuracy: 85.52%
57	Validation losses: 0.4192, 0.0914, 0.5105	Best Loss: 0.4663 (17)	Accuracy: 85.20%
58	Validation losses: 0.4235, 0.0907, 0.5143	Best Loss: 0.4663 (18)	Accuracy: 85.02%
59	Validation losses: 0.4355, 0.0907, 0.5262	Best Loss: 0.4663 (19)	Accuracy: 84.90%
60	Validation losses: 0.4720, 0.0916, 0.5636	Best Loss: 0.4663 (20)	Accuracy: 83.18%
Early stopping!
Total running time:	 4862
INFO:tensorflow:Restoring parameters from ./models/model_20240401174908.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    14.9500000477%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.2
Now: 20240401191011
0	Validation losses: 0.4397, 18.3486, 18.7883	Best Loss: 18.7883 (0)	Accuracy: 85.02%
1	Validation losses: 0.3824, 11.3217, 11.7041	Best Loss: 11.7041 (0)	Accuracy: 86.50%
2	Validation losses: 0.3886, 5.1068, 5.4954	Best Loss: 5.4954 (0)	A

49	Validation losses: 0.3577, 0.0841, 0.4418	Best Loss: 0.4330 (8)	Accuracy: 87.22%
50	Validation losses: 0.3618, 0.0845, 0.4463	Best Loss: 0.4330 (9)	Accuracy: 87.06%
51	Validation losses: 0.3621, 0.0846, 0.4467	Best Loss: 0.4330 (10)	Accuracy: 87.16%
52	Validation losses: 0.3592, 0.0842, 0.4434	Best Loss: 0.4330 (11)	Accuracy: 87.42%
53	Validation losses: 0.3688, 0.0842, 0.4530	Best Loss: 0.4330 (12)	Accuracy: 87.28%
54	Validation losses: 0.3745, 0.0858, 0.4603	Best Loss: 0.4330 (13)	Accuracy: 86.46%
55	Validation losses: 0.3768, 0.0845, 0.4613	Best Loss: 0.4330 (14)	Accuracy: 87.14%
56	Validation losses: 0.3828, 0.0842, 0.4670	Best Loss: 0.4330 (15)	Accuracy: 86.22%
57	Validation losses: 0.4816, 0.0983, 0.5798	Best Loss: 0.4330 (16)	Accuracy: 83.94%
58	Validation losses: 0.3736, 0.0885, 0.4621	Best Loss: 0.4330 (17)	Accuracy: 86.96%
59	Validation losses: 0.4065, 0.0869, 0.4934	Best Loss: 0.4330 (18)	Accuracy: 85.86%
60	Validation losses: 0.3945, 0.0877, 0.4822	Best Loss: 0.4330 (19)

43	Validation losses: 0.3525, 0.0804, 0.4329	Best Loss: 0.4254 (3)	Accuracy: 87.34%
44	Validation losses: 0.3511, 0.0804, 0.4315	Best Loss: 0.4254 (4)	Accuracy: 87.58%
45	Validation losses: 0.3577, 0.0809, 0.4386	Best Loss: 0.4254 (5)	Accuracy: 87.38%
46	Validation losses: 0.3577, 0.0806, 0.4383	Best Loss: 0.4254 (6)	Accuracy: 87.64%
47	Validation losses: 0.3616, 0.0813, 0.4429	Best Loss: 0.4254 (7)	Accuracy: 87.10%
48	Validation losses: 0.3703, 0.0812, 0.4515	Best Loss: 0.4254 (8)	Accuracy: 87.00%
49	Validation losses: 0.3777, 0.0808, 0.4585	Best Loss: 0.4254 (9)	Accuracy: 86.90%
50	Validation losses: 0.3684, 0.0814, 0.4498	Best Loss: 0.4254 (10)	Accuracy: 86.86%
51	Validation losses: 0.3656, 0.0811, 0.4467	Best Loss: 0.4254 (11)	Accuracy: 86.96%
52	Validation losses: 0.3680, 0.0822, 0.4501	Best Loss: 0.4254 (12)	Accuracy: 87.06%
53	Validation losses: 0.3790, 0.0824, 0.4614	Best Loss: 0.4254 (13)	Accuracy: 87.04%
54	Validation losses: 0.3625, 0.0822, 0.4448	Best Loss: 0.4254 (14)	Accu

...

18	Validation losses: 0.3329, 0.1934, 0.5263	Best Loss: 0.5062 (10)	Accuracy: 89.20%
19	Validation losses: 0.3335, 0.1914, 0.5248	Best Loss: 0.5062 (11)	Accuracy: 88.76%
20	Validation losses: 0.3733, 0.1893, 0.5626	Best Loss: 0.5062 (12)	Accuracy: 87.48%
21	Validation losses: 0.3318, 0.1868, 0.5186	Best Loss: 0.5062 (13)	Accuracy: 88.60%
22	Validation losses: 0.4031, 0.1838, 0.5869	Best Loss: 0.5062 (14)	Accuracy: 88.18%
23	Validation losses: 0.3725, 0.1808, 0.5533	Best Loss: 0.5062 (15)	Accuracy: 88.20%
24	Validation losses: 0.3170, 0.1778, 0.4948	Best Loss: 0.4948 (0)	Accuracy: 89.94%
25	Validation losses: 0.3713, 0.1748, 0.5461	Best Loss: 0.4948 (1)	Accuracy: 88.76%
26	Validation losses: 0.3650, 0.1722, 0.5372	Best Loss: 0.4948 (2)	Accuracy: 88.92%
27	Validation losses: 0.3341, 0.1695, 0.5036	Best Loss: 0.4948 (3)	Accuracy: 89.94%
28	Validation losses: 0.3498, 0.1668, 0.5166	Best Loss: 0.4948 (4)	Accuracy: 90.78%
29	Validation losses: 0.3437, 0.1645, 0.5082	Best Loss: 0.4948 (5)	Acc

3	Validation losses: 0.3682, 0.1422, 0.5104	Best Loss: 0.5017 (1)	Accuracy: 86.26%
4	Validation losses: 0.3263, 0.1421, 0.4684	Best Loss: 0.4684 (0)	Accuracy: 87.28%
5	Validation losses: 0.3412, 0.1420, 0.4832	Best Loss: 0.4684 (1)	Accuracy: 87.42%
6	Validation losses: 0.3323, 0.1418, 0.4741	Best Loss: 0.4684 (2)	Accuracy: 87.78%
7	Validation losses: 0.3800, 0.1417, 0.5217	Best Loss: 0.4684 (3)	Accuracy: 86.24%
8	Validation losses: 0.2849, 0.1415, 0.4264	Best Loss: 0.4264 (0)	Accuracy: 90.08%
9	Validation losses: 0.3197, 0.1414, 0.4612	Best Loss: 0.4264 (1)	Accuracy: 88.34%
10	Validation losses: 0.3450, 0.1414, 0.4864	Best Loss: 0.4264 (2)	Accuracy: 88.18%
11	Validation losses: 0.4244, 0.1413, 0.5657	Best Loss: 0.4264 (3)	Accuracy: 82.56%
12	Validation losses: 0.3414, 0.1412, 0.4826	Best Loss: 0.4264 (4)	Accuracy: 87.86%
13	Validation losses: 0.3237, 0.1411, 0.4648	Best Loss: 0.4264 (5)	Accuracy: 88.26%
14	Validation losses: 0.3251, 0.1410, 0.4661	Best Loss: 0.4264 (6)	Accuracy: 88.52%

35	Validation losses: 0.5548, 0.0985, 0.6533	Best Loss: 0.4040 (19)	Accuracy: 90.56%
36	Validation losses: 0.6121, 0.0982, 0.7103	Best Loss: 0.4040 (20)	Accuracy: 90.20%
Early stopping!
Total running time:	 3059
INFO:tensorflow:Restoring parameters from ./models/model_20240403114746.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    10.9800004959%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.6
Now: 20240403123847
0	Validation losses: 0.4294, 0.0899, 0.5193	Best Loss: 0.5193 (0)	Accuracy: 85.34%
1	Validation losses: 0.3508, 0.0900, 0.4408	Best Loss: 0.4408 (0)	Accuracy: 87.52%
2	Validation losses: 0.3449, 0.0901, 0.4350	Best Loss: 0.4350 (0)	Accuracy: 87.54%
3	Validation losses: 0.3999, 0.0902, 0.4901	Best Loss: 0.4350 (1)	Accuracy: 85.58%
4	Validation losses: 0.3225, 0.0904, 0.4129	Best Loss: 0.4129 (0)	Accuracy: 87.88%
5	Validation losses: 0.3278, 0.0906, 0.4184	Best Loss: 0.4129 (1)	Accuracy: 87.

28	Validation losses: 0.3931, 0.0700, 0.4631	Best Loss: 0.3621 (20)	Accuracy: 90.22%
Early stopping!
Total running time:	 2420
INFO:tensorflow:Restoring parameters from ./models/model_20240403135641.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.7900002003%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.8
Now: 20240403143703
0	Validation losses: 0.4304, 0.0567, 0.4872	Best Loss: 0.4872 (0)	Accuracy: 85.22%
1	Validation losses: 0.3534, 0.0568, 0.4102	Best Loss: 0.4102 (0)	Accuracy: 87.28%
2	Validation losses: 0.3548, 0.0569, 0.4118	Best Loss: 0.4102 (1)	Accuracy: 87.24%
3	Validation losses: 0.3723, 0.0571, 0.4294	Best Loss: 0.4102 (2)	Accuracy: 86.62%
4	Validation losses: 0.3262, 0.0573, 0.3835	Best Loss: 0.3835 (0)	Accuracy: 87.64%
5	Validation losses: 0.3194, 0.0576, 0.3770	Best Loss: 0.3770 (0)	Accuracy: 87.92%
6	Validation losses: 0.3377, 0.0579, 0.3956	Best Loss: 0.3770 (1)	Accuracy: 86.94

25	Validation losses: 0.3415, 0.0465, 0.3880	Best Loss: 0.3384 (19)	Accuracy: 89.76%
26	Validation losses: 0.3333, 0.0467, 0.3801	Best Loss: 0.3384 (20)	Accuracy: 90.84%
Early stopping!
Total running time:	 2252
INFO:tensorflow:Restoring parameters from ./models/model_20240403160010.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.6499998569%
RANDOM SEED:   3 	LAMBDA_EXP:   -7.0
Now: 20240403163744
0	Validation losses: 0.4237, 0.0358, 0.4595	Best Loss: 0.4595 (0)	Accuracy: 85.34%
1	Validation losses: 0.3560, 0.0359, 0.3918	Best Loss: 0.3918 (0)	Accuracy: 87.16%
2	Validation losses: 0.3566, 0.0360, 0.3926	Best Loss: 0.3918 (1)	Accuracy: 86.96%
3	Validation losses: 0.3690, 0.0361, 0.4051	Best Loss: 0.3918 (2)	Accuracy: 86.82%
4	Validation losses: 0.3237, 0.0363, 0.3600	Best Loss: 0.3600 (0)	Accuracy: 87.40%
5	Validation losses: 0.3344, 0.0365, 0.3709	Best Loss: 0.3600 (1)	Accuracy: 87.